# 🎂 Train Custom Cake AI Model + Export to ONNX

This notebook:
1. Trains a custom cake model using **LoRA** on SDXL
2. Exports to **ONNX format** for local/mobile deployment

## Setup:
1. **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order

---

In [ ]:
# Step 1: Install Required Libraries
!pip install -q diffusers transformers accelerate peft datasets huggingface_hub
!pip install -q onnx onnxruntime-gpu optimum[onnxruntime]
print("✅ Libraries installed!")

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login

print("🔐 Login to Hugging Face")
print("Get token from: https://huggingface.co/settings/tokens\n")

notebook_login()

In [ ]:
# Step 3: Load Your Cake Images Dataset
from datasets import load_dataset
import matplotlib.pyplot as plt

print("📥 Loading your cake images dataset...")
dataset = load_dataset("praveen9025/cake-images", split="train")
print(f"✅ Loaded {len(dataset)} cake images!")

# Preview
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    ax.imshow(dataset[i]['image'])
    ax.axis('off')
plt.show()

In [ ]:
# Step 4: Prepare Training Data
import os
from PIL import Image

TRAIN_DIR = "cake_training_data"
os.makedirs(TRAIN_DIR, exist_ok=True)

CAPTION = "a professional photo of cakestyle cake, bakery, studio lighting"

print("📁 Preparing training data...")
for i, item in enumerate(dataset):
    img = item['image'].convert('RGB').resize((512, 512), Image.LANCZOS)
    img.save(os.path.join(TRAIN_DIR, f"cake_{i:03d}.png"))
    with open(os.path.join(TRAIN_DIR, f"cake_{i:03d}.txt"), 'w') as f:
        f.write(CAPTION)

print(f"✅ Prepared {len(dataset)} images!")

In [ ]:
# Step 5: Clone Diffusers Training Scripts
!git clone https://github.com/huggingface/diffusers.git
!pip install -q ./diffusers
print("✅ Training scripts ready!")

In [ ]:
# Step 6: Train LoRA Model (30-60 minutes)
print("🚀 Starting training... ☕ Coffee break time!\n")

!accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir="cake_training_data" \
  --output_dir="cake-lora-model" \
  --instance_prompt="a photo of cakestyle cake" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --max_train_steps=500 \
  --rank=32 \
  --mixed_precision="fp16"

print("\n✅ Training complete!")

In [ ]:
# Step 7: Merge LoRA weights into base model (required for ONNX export)
from diffusers import DiffusionPipeline, AutoencoderKL
import torch

print("🔗 Merging LoRA weights into base model...")

# Load base model
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=torch.float16
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
).to("cuda")

# Load and merge LoRA
pipe.load_lora_weights("cake-lora-model")
pipe.fuse_lora()  # Merge LoRA into base weights

# Save merged model
MERGED_MODEL_PATH = "cake-model-merged"
pipe.save_pretrained(MERGED_MODEL_PATH)

print(f"✅ Merged model saved to: {MERGED_MODEL_PATH}")

In [ ]:
# Step 8: Export to ONNX Format! 🎉
from optimum.onnxruntime import ORTStableDiffusionXLPipeline

ONNX_OUTPUT_DIR = "cake-model-onnx"

print("📦 Exporting to ONNX format...")
print("⏱️ This takes 10-15 minutes...\n")

# Export using Optimum
onnx_pipe = ORTStableDiffusionXLPipeline.from_pretrained(
    MERGED_MODEL_PATH,
    export=True,
)

# Save ONNX model
onnx_pipe.save_pretrained(ONNX_OUTPUT_DIR)

print(f"\n✅ ONNX model saved to: {ONNX_OUTPUT_DIR}")
print(f"📁 Files created:")
!ls -la {ONNX_OUTPUT_DIR}

In [ ]:
# Step 9: Test ONNX Model
from optimum.onnxruntime import ORTStableDiffusionXLPipeline
import matplotlib.pyplot as plt

print("🧪 Testing ONNX model...")

# Load ONNX model
onnx_pipe = ORTStableDiffusionXLPipeline.from_pretrained(ONNX_OUTPUT_DIR)

# Generate test image
prompt = "a photo of cakestyle chocolate birthday cake with sprinkles"
image = onnx_pipe(prompt, num_inference_steps=20).images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title("ONNX Generated Cake")
plt.savefig("onnx_test_result.png")
plt.show()

print("✅ ONNX model working!")

In [ ]:
# Step 10: Download ONNX Model to Your Computer
import shutil

# Create zip file for easy download
ZIP_NAME = "cake-model-onnx.zip"

print(f"📦 Creating {ZIP_NAME}...")
shutil.make_archive("cake-model-onnx", 'zip', ONNX_OUTPUT_DIR)

print(f"\n✅ ONNX model packaged!")
print(f"📁 File size: {os.path.getsize(ZIP_NAME) / (1024*1024*1024):.2f} GB")
print(f"\n⬇️ Download 'cake-model-onnx.zip' from the Files panel on the left")

In [ ]:
# Step 11: (Optional) Save to Google Drive
from google.colab import drive

print("💾 Saving to Google Drive...")
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/cake-model-onnx"

# Copy to Drive
if os.path.exists(DRIVE_PATH):
    shutil.rmtree(DRIVE_PATH)
shutil.copytree(ONNX_OUTPUT_DIR, DRIVE_PATH)

# Also copy zip
shutil.copy(ZIP_NAME, "/content/drive/MyDrive/")

print(f"\n✅ ONNX model saved to Google Drive!")
print(f"📁 Location: Google Drive > cake-model-onnx")

---
## ✅ Done!

Your custom cake model is now in **ONNX format**!

### ONNX Files Created:
| File | Description |
|------|-------------|
| `text_encoder/model.onnx` | Text encoder |
| `unet/model.onnx` | Main diffusion model |
| `vae_decoder/model.onnx` | VAE decoder |
| `vae_encoder/model.onnx` | VAE encoder |

### Use Cases:
- **Local Python:** `ORTStableDiffusionXLPipeline.from_pretrained("cake-model-onnx")`
- **Mobile (Android):** Use ONNX Runtime Mobile
- **Web:** Use ONNX.js or Transformers.js

---